### Query for no. of records loaded into the database of all the data sets 

### Query for: Customer - Demography - transactions - item details 

The resultant of this query is stored in a file "amex_customer_demo_txn_item.csv"

### Query for merging training & testing dataset with campaign data 

The results are stored in "amex_train_campaign.csv" and "amex_test_campaign.csv"

### Script for merging the items that are valid for a coupon 

The resultant dataset is loaded into "amex_coupon_item_grouping.csv"

In [2]:
import pandas as pd
pd.options.display.html.table_schema = True

In [91]:
coupon_item_mapping = pd.read_csv('data//coupon_item_mapping.csv')
coupon_item_mapping.sample(5)

,coupon_id,item_id
10498,37,57131
26720,7,72803
10081,932,55367
69905,33,51288
40124,22,19975


In [92]:
coupons = set(coupon_item_mapping['coupon_id'].unique())

In [93]:
coupon_item_grouping = pd.DataFrame(columns={'items','coupon_id'})

In [94]:
counter = 0
for c in coupons:
    items_for_coupon = list(set(coupon_item_mapping[coupon_item_mapping.coupon_id == c]['item_id']))
    coupon_item_grouping.at[counter,'items'] = items_for_coupon
    coupon_item_grouping.at[counter,'coupon_id'] = c
    counter += 1
coupon_item_grouping.shape

(1116, 2)

In [95]:
coupon_item_grouping.sample(5)

,items,coupon_id
608,"[7813, 39560, 4238, 44305, 57107, 57108, 43029...",609
1110,"[20337, 29370, 31498, 18164]",1111
118,"[38178, 7106, 40005, 42775, 51602, 31442, 3102...",119
714,"[9889, 20803, 10116, 14731, 11628, 39308, 1281...",715
479,[72679],480


In [96]:
coupon_item_grouping.to_csv('data//amex_coupon_item_grouping.csv',index=False)

### Merging the above coupon_item_grouping to the amex_train_campaign and amex_test_campaign datasets

In [97]:
amex_train_campaign = pd.read_csv('data//amex_train_campaign.csv',infer_datetime_format=True,parse_dates=True)
amex_test_campaign = pd.read_csv('data//amex_test_campaign.csv',infer_datetime_format=True,parse_dates=True)
amex_train_campaign.columns = ['id','customer_id','campaign_id','campaign_type','start_date','end_date','coupon_id','redumption_status']
amex_test_campaign.columns = ['id','customer_id','campaign_id','campaign_type','start_date','end_date','coupon_id','redumption_status']
amex_train_campaign.shape, amex_test_campaign.shape

((78369, 8), (50226, 8))

In [98]:
amex_train_campaign.sample(5)

,id,customer_id,campaign_id,campaign_type,start_date,end_date,coupon_id,redumption_status
16655,27404,1429,30,X,19-NOV-12,04-JAN-13,595,0
72841,119640,34,13,X,19-MAY-13,05-JUL-13,23,0
74094,121680,466,8,X,16-FEB-13,05-APR-13,672,0
74903,123001,474,29,Y,08-OCT-12,30-NOV-12,445,0
31553,51708,1482,8,X,16-FEB-13,05-APR-13,579,0


In [99]:
amex_test_campaign.sample(5)

,id,customer_id,campaign_id,campaign_type,start_date,end_date,coupon_id,redumption_status
32161,82425,302,18,X,10-AUG-13,04-OCT-13,856,NaN
3250,8063,844,20,Y,07-SEP-13,16-NOV-13,836,NaN
37400,95534,1095,22,X,16-SEP-13,18-OCT-13,518,NaN
25603,65465,1270,20,Y,07-SEP-13,16-NOV-13,263,NaN
2040,5131,247,16,Y,15-JUL-13,16-AUG-13,28,NaN


In [100]:
amex_train_campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78369 entries, 0 to 78368
Data columns (total 8 columns):
id                   78369 non-null int64
customer_id          78369 non-null int64
campaign_id          78369 non-null int64
campaign_type        78369 non-null object
start_date           78369 non-null object
end_date             78369 non-null object
coupon_id            78369 non-null int64
redumption_status    78369 non-null int64
dtypes: int64(5), object(3)
memory usage: 4.8+ MB


In [101]:
amex_train_campaign_coupon = pd.merge(amex_train_campaign,coupon_item_grouping,on="coupon_id")
amex_test_campaign_coupon = pd.merge(amex_test_campaign,coupon_item_grouping,on="coupon_id")

In [102]:
amex_train_campaign_coupon.sample(5)

,id,customer_id,campaign_id,campaign_type,start_date,end_date,coupon_id,redumption_status,items
1971,78751,575,9,Y,11-MAR-13,12-APR-13,705,0,"[39436, 73742, 13841, 24085, 62998, 73239, 732..."
42342,98290,515,13,X,19-MAY-13,05-JUL-13,653,0,"[71238, 25225, 71186, 71190, 72443]"
16717,101658,124,8,X,16-FEB-13,05-APR-13,544,0,"[40450, 21666, 31076, 47107, 29865, 14315, 178..."
41930,69263,432,13,X,19-MAY-13,05-JUL-13,979,0,"[49896, 56842, 58667]"
24952,49008,499,7,Y,02-FEB-13,08-MAR-13,1050,0,"[64832, 64954]"


In [103]:
amex_test_campaign_coupon.sample(5)

,id,customer_id,campaign_id,campaign_type,start_date,end_date,coupon_id,redumption_status,items
33348,49576,431,25,Y,21-OCT-13,22-NOV-13,522,NaN,"[5634, 65028, 56871, 41522, 35894, 42046, 6731..."
18761,105782,935,22,X,16-SEP-13,18-OCT-13,432,NaN,"[41346, 36099, 45061, 5126, 1415, 46853, 44939..."
38471,85970,934,18,X,10-AUG-13,04-OCT-13,387,NaN,"[41601, 49409, 54664, 40587, 42131, 42517, 640..."
9041,29081,196,20,Y,07-SEP-13,16-NOV-13,841,NaN,"[1543, 17417, 5139, 48673, 48679, 48685, 4147,..."
15481,121283,585,22,X,16-SEP-13,18-OCT-13,801,NaN,"[24839, 264, 8457, 12430, 46997, 45338, 34076,..."


In [104]:
amex_train_campaign_coupon.to_csv('data//amex_train_campaign_coupon.csv',index=False)
amex_test_campaign_coupon.to_csv('data//amex_test_campaign_coupon.csv',index=False)

In [105]:
del(amex_train_campaign,amex_test_campaign,coupon_item_grouping,coupon_item_mapping,counter, coupons)

In [117]:
amex_customer_demo_txn_item = pd.read_csv('data//amex_customer_demo_txn_item.csv',infer_datetime_format=True,parse_dates=True)
cols = [i.lower() for i in list(amex_customer_demo_txn_item.columns)]
amex_customer_demo_txn_item.columns = cols
amex_customer_demo_txn_item.sample(5)

,customer_id,date_of_txn,item_id,brand,brand_type,category,quantity,selling_price,other_discount,coupon_discount,age_range,marital_status,rented,family_size,no_of_children,income_bracket
370984,1431,27-OCT-12,58476,686,Established,Pharmaceutical,1,373.65,-53.43,0.0,46-55,Married,0,2,NaN,12
140134,128,07-JUN-12,14440,1933,Established,Grocery,1,59.49,-47.02,0.0,36-45,Married,0,3,1,1
15011,1464,20-FEB-12,11103,1278,Established,Packaged Meat,2,70.53,-85.49,0.0,18-25,NaN,0,1,NaN,1
439422,1136,06-DEC-12,20308,1143,Established,Grocery,1,35.62,-7.84,0.0,46-55,Married,0,2,NaN,6
485969,78,03-JAN-13,31890,56,Local,Packaged Meat,1,248.98,-60.55,0.0,36-45,Married,0,4,2,8


In [ ]:
alpha = pd.DataFrame()
alpha_list = []
for i in range(len(amex_customer_demo_txn_item)):
    c_id = amex_customer_demo_txn_item.loc[i,'customer_id']
    d_txn = amex_customer_demo_txn_item.loc[i,'date_of_txn']
    i_id = amex_customer_demo_txn_item.loc[i,'item_id']
    beta = amex_train_campaign_coupon[amex_train_campaign_coupon.customer_id==c_id][['campaign_id','start_date','end_date']].reset_index()
    for j in range(len(beta)):
        strt = beta.loc[j,'start_date']
        end = beta.loc[j,'end_date']
        if (d_txn > strt) and (d_txn <= end):
            alpha_list.append('Y')
        else: 
            alpha_list.append('N')

In [ ]:
alpha_list.count('Y'),alpha_list.count('N')

## Reference

### DDL Statements for Loading Datasets into Database